#Simulated Annealing Using Class

In [10]:
from random import *
from math import *
from time import time

In [11]:
# Reaches in a single step
start_state = [1,2,3,4,5,6,7,0,8]

# Does not necessarily reach
# start_state = [4,5,1,2,3,8,0,6,7]

target_state = [1,2,3,4,5,6,7,8,0]
heuristics = {0:"Misplaced Tiles",1:"Manhattan Distance"}


In [12]:

def get_row(pos):
        return (int(pos/3))

def get_col(pos):
        return int(pos%3)

def get_index(row, col):
        return row*3+col

    
class Node:

    goal_state = None
    start_state = None
    hn = None                              
        
    def __init__(self,state,parent,action):     
        
        self.state = state
        self.parent = parent
        self.action = action

   
    def all_pos_moves(self):
        blank_pos = self.state.index(0) 
        r = get_row(blank_pos)
        c = get_col(blank_pos)
        moves = ["U", "D", "L", "R"]
        if r == 0:
            moves.remove("U")
        if r == 2:
            moves.remove("D")
        if c == 0:
            moves.remove("L")
        if c == 2:
            moves.remove("R")
        return moves

    def do_move(self,move):
        new_state = self.state.copy()
        blank_pos = new_state.index(0)
        r = get_row(blank_pos)
        c = get_col(blank_pos)
        if move == "U":
            move_index = get_index(r-1, c)
        if move == "D":
            move_index = get_index(r+1, c)
        if move == "R":
            move_index = get_index(r, c+1)
        if move == "L":
            move_index = get_index(r, c-1)
        new_state[blank_pos],new_state[move_index] = new_state[move_index],new_state[blank_pos]
        return Node(new_state,self,move)

    def cal_e(self):
        
        md = 0
        mt = 0
        
        for i in range(9):
            
            if self.state[i] == 0:
                continue
            else:
                md += abs(self.state[i] - self.goal_state[i])
            
            if(self.state[i] != self.goal_state[i]): mt += 1
                
        
        if self.hn == 1:
            return md
        elif self.hn == 2:
            return mt
        else:
            return (md*mt)

        
    def Is_solved(self):
        if(self.state == self.goal_state):
            return True
        else:
            return False
    
    def trace_path(self):                    
        
        solution_list = []
        solution_list.append(self.action)            
        path = self
        while path.parent != None:              
            path = path.parent         
            solution_list.append(path.action)
        solution_list = solution_list[:-1]
        solution_list.reverse()
        return solution_list
    


In [13]:
def is_grid_valid(input_grid):
  count_inversion = 0;
  grid_length = 9;
  if input_grid[-1] == 0:
    grid_length = 8;
    for i in range(7):
      for j in range(i+1,8):
        if input_grid[i] == 0 or input_grid[j] == 0:
          continue;
        if (int(input_grid[i]) > int(input_grid[j])):
          count_inversion = count_inversion+1;
  return ((count_inversion%2) == 0);  

In [14]:
def print_grid(current_grid):
  ctr = 1
  for i in current_grid:
    print(i,end = '   ')
    ctr = ctr+1
    if(ctr == 4 or ctr == 7):
      print('\n')

In [15]:
def sim_ann(initial_state,goal_state, temp, hn):

    Node.start_state = initial_state
    Node.goal_state = goal_state
    Node.hn = hn
    temperature = temp
    state = Node(initial_state,None,None)   
    
    oldE = state.cal_e()                     
    n_explored = 0
    flag =False                              

    while temperature > 0:
        moves = state.all_pos_moves()             
        accepted = False
        
        while not accepted:

            m = choice(moves)              
            new_state = state.do_move(m)   
            n_explored+=1                  
            newE = new_state.cal_e()      
            
            deltaE = newE - oldE           
            if deltaE <= 0:
                accepted = True
                state = new_state          
            else:
                boltz = exp(-float(deltaE)/temperature)
                r = random()                          
                if r <= boltz:                         
                    accepted = True
                    state = new_state
        
        oldE = newE                         
        temperature = cool(temperature)    
        
        if state.Is_solved():                  
            flag = True
            break;
    
    optimal_path = state.trace_path()           
    return(optimal_path,n_explored,flag)

In [16]:
h_1 = 'Misplaced Tiles'
h_2 = 'Manhattan Distance'
h_3 = "h3 = h1 * h2"
heuristics[h_3] = 3

In [17]:
temp = 100

cool_f = 'T = T-1'
def cool(T):
    return (T-1)

In [18]:

print("Start State")
print_grid(start_state)
print("\nGoal State")
print_grid(target_state)


if(is_grid_valid(start_state)):
  print("\nGrid is Solvable :)")
else:
  print("\nGrid is Unsovlable :(")

for hi in heuristics:
    print("\nHeuristics chosen:",heuristics[hi],"\nTemperature chosen:",temp,"\nCooling function:",cool_f,"\n")
    
    start_epoch = time()
    output = sim_ann(start_state, target_state, temp, heuristics[hi]) 
    exe_time = time() - start_epoch

    optimal_path = output[0]
    n_state = output[1]
    flag = output[2]

    if flag:
        print("Puzzle solved successfully\n")
        print('Optimal path:',optimal_path)

    else:
        print("Puzzle cant be solved\n")

    print('Total no. of states explored: ',n_state)
    print('\nTime taken: ', exe_time)
    print("\n\n")

Start State
1   2   3   

4   5   6   

7   0   8   
Goal State
1   2   3   

4   5   6   

7   8   0   
Grid is Solvable :)

Heuristics chosen: Misplaced Tiles 
Temperature chosen: 100 
Cooling function: T = T-1 

Puzzle solved successfully

Optimal path: ['L', 'U', 'D', 'U', 'R', 'L', 'U', 'D', 'U', 'D', 'D', 'R', 'R']
Total no. of states explored:  14

Time taken:  0.0002715587615966797




Heuristics chosen: Manhattan Distance 
Temperature chosen: 100 
Cooling function: T = T-1 

Puzzle solved successfully

Optimal path: ['R']
Total no. of states explored:  1

Time taken:  4.601478576660156e-05




Heuristics chosen: 3 
Temperature chosen: 100 
Cooling function: T = T-1 

Puzzle solved successfully

Optimal path: ['L', 'R', 'R']
Total no. of states explored:  3

Time taken:  7.414817810058594e-05



